## 데이터 및 라이브러리 로딩

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

train = pd.read_csv('movies_train.csv')
test = pd.read_csv('movies_test.csv')

## 라이브러리

In [2]:
#배급사 전처리
def get_dis(x) :
    if 'CJ' in x or 'CGV' in x :
        return 'CJ'
    elif '쇼박스' in x :
        return '쇼박스'
    elif 'SK' in x :
        return 'SK'
    elif '리틀빅픽' in x :
        return '리틀빅픽처스'
    elif '스폰지' in x :
        return '스폰지'
    elif '싸이더스' in x :
        return '싸이더스'
    elif '에이원' in x :
        return '에이원'
    elif '마인스' in x :
        return '마인스'
    elif '마운틴픽' in x :
        return '마운틴픽처스'
    elif '디씨드' in x :
        return '디씨드'
    elif '드림팩트' in x :
        return '드림팩트'
    elif '메가박스' in x :
        return '메가박스'
    elif '마운틴' in x :
        return '마운틴'
    else :
        return x

In [3]:
#개봉계절 전처리
def get_month(x):
    if x[5:7] == '12' or x[5:7] == '01' or x[5:7] =='02':
        return 1
    elif x[5:7] == '03' or x[5:7] =='04' or x[5:7] =='05':
        return 2
    elif x[5:7] == '06' or x[5:7] =='07' or x[5:7] =='08':
        return 3
    elif x[5:7] == '09' or x[5:7] =='10' or x[5:7] =='11':
        return 4

## 데이터 전처리

In [4]:
#배급사 이름 정리
train['distributor'] = train.distributor.apply(get_dis)
test['distributor'] = test.distributor.apply(get_dis)

In [5]:
#장르 중위값 계산
train.groupby('genre').box_off_num.mean().sort_values()

genre
뮤지컬       6.627000e+03
다큐멘터리     6.717226e+04
서스펜스      8.261100e+04
애니메이션     1.819267e+05
멜로/로맨스    4.259680e+05
미스터리      5.275482e+05
공포        5.908325e+05
드라마       6.256898e+05
코미디       1.193914e+06
SF        1.788346e+06
액션        2.203974e+06
느와르       2.263695e+06
Name: box_off_num, dtype: float64

In [6]:
#장르 순위별로 랭크1
train['genre_rank'] = train.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})
test['genre_rank'] = test.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})

In [7]:
#배급사 순위별로 랭크1
tr_nm_rank = train.groupby('distributor').box_off_num.median().reset_index(name = 'num_rank').sort_values(by = 'num_rank')

In [8]:
#배급사 순위별로 랭크2
tr_nm_rank['num_rank'] = [i + 1 for i in range(tr_nm_rank.shape[0])]
tr_nm_rank

,distributor,num_rank
124,인피니티엔터테인먼트,1
51,고구마공작소,2
78,사람과 사람들,3
114,위드시네마,4
54,나우콘텐츠,5
...,...,...
126,전망좋은영화사,148
119,이십세기폭스코리아(주),149
82,쇼박스,150
104,영구아트무비,151


In [9]:
#개봉일 구분
time_month = train.release_time.apply(get_month)
train['time2'] = time_month
train

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,genre_rank,time2
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398,11,4
1,내부자들,쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,12,4
2,은밀하게 위대하게,쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083,11,3
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866,9,3
4,불량남녀,쇼박스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387,9,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,해무,(주)NEW,드라마,2014-08-13,111,청소년 관람불가,심성보,3833.00,1,510,7,1475091,8,3
596,파파로티,쇼박스,드라마,2013-03-14,127,15세 관람가,윤종찬,496061.00,1,286,6,1716438,8,2
597,살인의 강,마운틴픽처스,공포,2010-09-30,99,청소년 관람불가,김대현,NaN,0,123,4,2475,7,4
598,악의 연대기,CJ,느와르,2015-05-14,102,15세 관람가,백운학,NaN,0,431,4,2192525,12,2


In [10]:
#개봉일 구분
time_month = test.release_time.apply(get_month)
test['time2'] = time_month
test

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_rank,time2
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3,12,1
1,아빠가 여자를 좋아해,쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3,5,1
2,하모니,CJ,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7,8,1
3,의형제,쇼박스,액션,2010-02-04,116,15세 관람가,장훈,6.913420e+05,2,408,2,11,1
4,평행 이론,CJ,공포,2010-02-18,110,15세 관람가,권호영,3.173800e+04,1,380,1,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,해에게서 소년에게,디씨드,드라마,2015-11-19,78,15세 관람가,안슬기,2.590000e+03,1,4,4,8,4
239,울보 권투부,인디스토리,다큐멘터리,2015-10-29,86,12세 관람가,이일하,NaN,0,18,2,2,4
240,어떤살인,(주)컨텐츠온미디어,느와르,2015-10-28,107,청소년 관람불가,안용훈,NaN,0,224,4,12,4
241,말하지 못한 비밀,마운틴픽처스,드라마,2015-10-22,102,청소년 관람불가,송동윤,5.069900e+04,1,68,7,8,4


In [11]:
#병합
train = pd.merge(train, tr_nm_rank, how = 'left')
test = pd.merge(test, tr_nm_rank, how = 'left')

In [12]:
train

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,genre_rank,time2,num_rank
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398,11,4,138
1,내부자들,쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,12,4,150
2,은밀하게 위대하게,쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083,11,3,150
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866,9,3,145
4,불량남녀,쇼박스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387,9,4,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,해무,(주)NEW,드라마,2014-08-13,111,청소년 관람불가,심성보,3833.00,1,510,7,1475091,8,3,145
596,파파로티,쇼박스,드라마,2013-03-14,127,15세 관람가,윤종찬,496061.00,1,286,6,1716438,8,2,150
597,살인의 강,마운틴픽처스,공포,2010-09-30,99,청소년 관람불가,김대현,NaN,0,123,4,2475,7,4,47
598,악의 연대기,CJ,느와르,2015-05-14,102,15세 관람가,백운학,NaN,0,431,4,2192525,12,2,146


In [13]:
test

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_rank,time2,num_rank
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3,12,1,136.0
1,아빠가 여자를 좋아해,쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3,5,1,150.0
2,하모니,CJ,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7,8,1,146.0
3,의형제,쇼박스,액션,2010-02-04,116,15세 관람가,장훈,6.913420e+05,2,408,2,11,1,150.0
4,평행 이론,CJ,공포,2010-02-18,110,15세 관람가,권호영,3.173800e+04,1,380,1,7,1,146.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,해에게서 소년에게,디씨드,드라마,2015-11-19,78,15세 관람가,안슬기,2.590000e+03,1,4,4,8,4,NaN
239,울보 권투부,인디스토리,다큐멘터리,2015-10-29,86,12세 관람가,이일하,NaN,0,18,2,2,4,72.0
240,어떤살인,(주)컨텐츠온미디어,느와르,2015-10-28,107,청소년 관람불가,안용훈,NaN,0,224,4,12,4,NaN
241,말하지 못한 비밀,마운틴픽처스,드라마,2015-10-22,102,청소년 관람불가,송동윤,5.069900e+04,1,68,7,8,4,47.0


# 여기서부터 해야하는건 드랍하고 나머지 원핫인코딩한 다음에 돌리는 것, 근데 참고할건 이미 밑에 전체를 원핫인코딩하기 때문에 조금 더 잘 봐야할 것 같아

In [12]:
# 결측치가 많은 데이터 제거
train = train.drop(['release_time', 'dir_prev_bfnum', 'director', 'title',  'dir_prev_num', 'screening_rat', 'time', 'distributor' , 'genre', 'num_actor'],axis = 1)
test =  test.drop(['release_time', 'dir_prev_bfnum', 'director', 'title',  'dir_prev_num', 'screening_rat', 'time', 'distributor' , 'genre', 'num_actor'],axis = 1)

In [13]:
train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

In [14]:
test

,num_staff,genre_rank,time2,num_rank
0,304,12,1,136.0
1,275,5,1,150.0
2,419,8,1,146.0
3,408,11,1,150.0
4,380,7,1,146.0
...,...,...,...,...
238,4,8,4,0.0
239,18,2,4,72.0
240,224,12,4,0.0
241,68,8,4,47.0


In [15]:
# 원핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

## 모델정의 및 학습

In [16]:
train_x = train.drop(['box_off_num'],axis= 1)
train_y = train['box_off_num']

In [17]:
model=RandomForestRegressor(n_estimators=100)
model.fit(train_x,train_y)

RandomForestRegressor()

# 학습 된 모델로 예측 데이터 생성

In [18]:
pred = model.predict(test)

## 제출파일 생성

In [19]:
submission = pd.read_csv('submission.csv')
submission

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0
...,...,...
238,해에게서 소년에게,0
239,울보 권투부,0
240,어떤살인,0
241,말하지 못한 비밀,0


In [20]:
submission['box_off_num'] = pred

In [21]:
submission

,title,box_off_num
0,용서는 없다,1.873312e+06
1,아빠가 여자를 좋아해,1.549690e+06
2,하모니,2.263116e+06
3,의형제,4.173649e+06
4,평행 이론,1.936187e+06
...,...,...
238,해에게서 소년에게,9.197000e+01
239,울보 권투부,7.250507e+03
240,어떤살인,1.838538e+05
241,말하지 못한 비밀,2.680180e+03


In [22]:
submission.to_csv('suhykii_first_movies.csv',index = False)

In [23]:
from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'suhykii_first_movies.csv', 
'61bbe3d0a7cc90dea2a88fe1a617507b8b597f9bdb5e200bb3a039208649ae7d', 
'235536', 
'수키', 
'RandomForestRegressor' )


{'isSubmitted': True, 'detail': 'Success'}
